In [1]:
import pandas as pd
from scipy import stats
import scikit_posthocs as sp
from statsmodels import stats as sm
import numpy as np

# Skin microbiome

In [2]:
# load metadata skin
metadata_skin = pd.read_csv("/Users/dagmarschierova/MEGA/MBÚ/swabs_ZS/inputs/metadata_swabs.tsv", sep='\t')
metadata_skin = metadata_skin.iloc[1:]
metadata_skin.set_index('#SampleID', inplace=True)

# convert numeric columns to float
metadata_skin['Age'] = metadata_skin['Age'].astype(float)
metadata_skin['BMI'] = metadata_skin['BMI'].astype(float)
metadata_skin['CRP'] = metadata_skin['CRP'].astype(float)
metadata_skin['WBC'] = metadata_skin['WBC'].astype(float)
metadata_skin['PLT'] = metadata_skin['PLT'].astype(float)
metadata_skin['Ferritin'] = metadata_skin['Ferritin'].astype(float)
metadata_skin['Hb'] = metadata_skin['Hb'].astype(float)
metadata_skin['FC'] = metadata_skin['FC'].astype(float)

# filter patients on UST
ust_skin = metadata_skin[metadata_skin["Drug"]=="UST"]
ust_skin.drop(labels=["P8_LV78_I_D", "P11_GT85_VI_D", "P36_KJ78_V_D", "P43_BR63_II_D"], axis=0, inplace=True)

#convert numeric columns to float
ust_skin['Disease_duration_years'] = ust_skin['Disease_duration_years'].astype(float)
ust_skin['Age_at_diagnosis'] = ust_skin['Age_at_diagnosis'].astype(float)

# filter out HC
ust_skin_hc = metadata_skin[(metadata_skin["Drug"]=="HC") & (metadata_skin["Sampling_site"]=="D")]
ust_skin_hc.drop(labels=["K40_WV86_I_D", "K31_MM80_I_D", "K31_MM80_IV_D"], axis=0, inplace=True)


/Users/dagmarschierova/MEGA/MBÚ/venv_MBU/lib/python3.9/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-2-05812302ce1f>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_skin['Disease_duration_years'] = ust_skin['Disease_duration_years'].astype(float)
<ipython-input-2-05812302ce1f>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [3]:
# baseline
ust_skin_cd_0 = ust_skin[(ust_skin["Dg"]=="CD") & (ust_skin["Week"]=="week_0")]
ust_skin_uc_0 = ust_skin[(ust_skin["Dg"]=="UC") & (ust_skin["Week"]=="week_0")]
ust_skin_0 = ust_skin[((ust_skin["Dg"]=="CD") | (ust_skin["Dg"]=="UC")) & (ust_skin["Week"]=="week_0")]

# all patients present visit 3
ust_skin_cd_3 = ust_skin[(ust_skin["Dg"]=="CD") & (ust_skin["Week"]=="week_16")]
ust_skin_uc_3 = ust_skin[(ust_skin["Dg"]=="UC") & (ust_skin["Week"]=="week_16")]
ust_skin_3 = ust_skin[((ust_skin["Dg"]=="CD") | (ust_skin["Dg"]=="UC")) & (ust_skin["Week"]=="week_16")]

# endpoint
ust_skin_cd_40 = ust_skin[(ust_skin["Dg"]=="CD") & (ust_skin["Week"]=="week_40")]
ust_skin_uc_40 = ust_skin[(ust_skin["Dg"]=="UC") & (ust_skin["Week"]=="week_40")]
ust_skin_40 = ust_skin[((ust_skin["Dg"]=="CD") | (ust_skin["Dg"]=="UC")) & (ust_skin["Week"]=="week_40")]

# healthy controls
ust_skin_hc_0 = ust_skin_hc[ust_skin_hc["Week"]=="week_0"]
ust_skin_hc_0 = ust_skin_hc_0.append(ust_skin_hc.loc["K40_WV86_IV_D"])

In [4]:
print("Number of IBD patients at baseline (week 0) = ", ust_skin[(ust_skin['Health_status']=="patient") & (ust_skin['Week']=="week_0")]["Patient"].nunique())
print("Number of IBD patients at endpoint (week 40) = ", ust_skin[(ust_skin['Health_status']=="patient") & (ust_skin['Week']=="week_40")]["Patient"].nunique())
print("Number of IBD females at baseline (week 0) = ", ust_skin[(ust_skin['Sex']=="F") & (ust_skin['Week']=="week_0")]["Patient"].nunique())
print("Number of IBD males at baseline (week 0) = ", ust_skin[(ust_skin['Sex']=="M") & (ust_skin['Week']=="week_0")]["Patient"].nunique())
print("Number of IBD females at endpoint (week 40) = ", ust_skin[(ust_skin['Sex']=="F") & (ust_skin['Week']=="week_40")]["Patient"].nunique())
print("Number of IBD males at endpoint (week 40) = ", ust_skin[(ust_skin['Sex']=="M") & (ust_skin['Week']=="week_40")]["Patient"].nunique())
print("Number of CD patients", ust_skin[(ust_skin['Dg']=="CD")]["Patient"].nunique())
print("Number of UC patients", ust_skin[(ust_skin['Dg']=="UC")]["Patient"].nunique())
print("Number of CD patients at baseline (week 0)", ust_skin[(ust_skin['Dg']=="CD") & (ust_skin['Week']=="week_0")]["Patient"].nunique())
print("Number of UC patients at baseline (week 0)", ust_skin[(ust_skin['Dg']=="UC") & (ust_skin['Week']=="week_0")]["Patient"].nunique())
print("Number of CD patients at endpoint (week 40)", ust_skin[(ust_skin['Dg']=="CD") & (ust_skin['Week']=="week_40")]["Patient"].nunique())
print("Number of UC patients at endpoint (week 40)", ust_skin[(ust_skin['Dg']=="UC") & (ust_skin['Week']=="week_40")]["Patient"].nunique())
print("Number of remitters at baseline (week 0) = ", ust_skin[(ust_skin['Disease_severity_scale4']=="remission") & (ust_skin['Week']=="week_0")]["Patient"].nunique())
print("Number of remitters at endpoint (week 40) = ", ust_skin[(ust_skin['Disease_severity_scale4']=="remission") & (ust_skin['Week']=="week_40")]["Patient"].nunique())
print("Number of HC = ", ust_skin_hc[ust_skin_hc['Health_status']=="healthy_control"]["Patient"].nunique())
print("Number of HC females entering the study = ", ust_skin_hc[(ust_skin_hc['Sex']=="F")]["Patient"].nunique())
print("Number of HC males entering the study = ", ust_skin_hc[(ust_skin_hc['Sex']=="M")]["Patient"].nunique())

Number of IBD patients at baseline (week 0) =  8
Number of IBD patients at endpoint (week 40) =  5
Number of IBD females at baseline (week 0) =  4
Number of IBD males at baseline (week 0) =  4
Number of IBD females at endpoint (week 40) =  2
Number of IBD males at endpoint (week 40) =  3
Number of CD patients 7
Number of UC patients 2
Number of CD patients at baseline (week 0) 6
Number of UC patients at baseline (week 0) 2
Number of CD patients at endpoint (week 40) 4
Number of UC patients at endpoint (week 40) 1
Number of remitters at baseline (week 0) =  3
Number of remitters at endpoint (week 40) =  4
Number of HC =  22
Number of HC females entering the study =  14
Number of HC males entering the study =  8


In [5]:
## Anthropometric parameters
# CD all patients visit
print(f'CD patients Age quantile\n{ust_skin_cd_3.Age.quantile([0.25,0.5,0.75])}')
print(f'CD patients BMI quantile\n{ust_skin_cd_3.BMI.quantile([0.25,0.5,0.75])}')
print(f'CD patients Disease duration years quantile\n{ust_skin_cd_3.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'CD patients Age at diagnosis quantile\n{ust_skin_cd_3.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')

# UC all patients visit
print(f'UC patients Age quantile\n{ust_skin_uc_3.Age.quantile([0.25,0.5,0.75])}')
print(f'UC patients BMI quantile\n{ust_skin_uc_3.BMI.quantile([0.25,0.5,0.75])}')
print(f'UC patients Disease duration years quantile\n{ust_skin_uc_3.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'UC patients Age at diagnosis quantile\n{ust_skin_uc_3.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')

# IBD all patients visit
print(f'IBD patients Age quantile\n{ust_skin_3.Age.quantile([0.25,0.5,0.75])}')
print(f'IBD patients BMI quantile\n{ust_skin_3.BMI.quantile([0.25,0.5,0.75])}')
print(f'IBD patients Disease duration years quantile\n{ust_skin_3.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'IBD patients Age at diagnosis quantile\n{ust_skin_3.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')

CD patients Age quantile
0.25    40.0
0.50    43.0
0.75    50.5
Name: Age, dtype: float64
CD patients BMI quantile
0.25    25.7800
0.50    26.7850
0.75    28.8925
Name: BMI, dtype: float64
CD patients Disease duration years quantile
0.25     4.5
0.50    10.0
0.75    12.5
Name: Disease_duration_years, dtype: float64
CD patients Age at diagnosis quantile
0.25    29.5
0.50    37.0
0.75    41.0
Name: Age_at_diagnosis, dtype: float64
UC patients Age quantile
0.25    40.5
0.50    48.0
0.75    55.5
Name: Age, dtype: float64
UC patients BMI quantile
0.25    20.3575
0.50    22.0250
0.75    23.6925
Name: BMI, dtype: float64
UC patients Disease duration years quantile
0.25    11.25
0.50    12.50
0.75    13.75
Name: Disease_duration_years, dtype: float64
UC patients Age at diagnosis quantile
0.25    31.0
0.50    37.0
0.75    43.0
Name: Age_at_diagnosis, dtype: float64
IBD patients Age quantile
0.25    40.0
0.50    43.0
0.75    55.0
Name: Age, dtype: float64
IBD patients BMI quantile
0.25    23.912

In [6]:
## Anthropometric parameters
# CD patients baseline
print(f'CD patients Age quantile\n{ust_skin_cd_0.Age.quantile([0.25,0.5,0.75])}')
print(f'CD patients BMI quantile\n{ust_skin_cd_0.BMI.quantile([0.25,0.5,0.75])}')
print(f'CD patients Disease duration years quantile\n{ust_skin_cd_0.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'CD patients Age at diagnosis quantile\n{ust_skin_cd_0.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')

# UC patients baseline
print(f'UC patients Age quantile\n{ust_skin_uc_0.Age.quantile([0.25,0.5,0.75])}')
print(f'UC patients BMI quantile\n{ust_skin_uc_0.BMI.quantile([0.25,0.5,0.75])}')
print(f'UC patients Disease duration years quantile\n{ust_skin_uc_0.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'UC patients Age at diagnosis quantile\n{ust_skin_uc_0.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')

# IBD patients baseline
print(f'IBD patients Age quantile\n{ust_skin_0.Age.quantile([0.25,0.5,0.75])}')
print(f'IBD patients BMI quantile\n{ust_skin_0.BMI.quantile([0.25,0.5,0.75])}')
print(f'IBD patients Disease duration years quantile\n{ust_skin_0.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'IBD patients Age at diagnosis quantile\n{ust_skin_0.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')

CD patients Age quantile
0.25    40.75
0.50    44.50
0.75    52.75
Name: Age, dtype: float64
CD patients BMI quantile
0.25    24.985
0.50    27.360
0.75    27.620
Name: BMI, dtype: float64
CD patients Disease duration years quantile
0.25     2.75
0.50     9.50
0.75    13.25
Name: Disease_duration_years, dtype: float64
CD patients Age at diagnosis quantile
0.25    29.5
0.50    38.0
0.75    42.0
Name: Age_at_diagnosis, dtype: float64
UC patients Age quantile
0.25    40.5
0.50    48.0
0.75    55.5
Name: Age, dtype: float64
UC patients BMI quantile
0.25    19.315
0.50    21.330
0.75    23.345
Name: BMI, dtype: float64
UC patients Disease duration years quantile
0.25    11.25
0.50    12.50
0.75    13.75
Name: Disease_duration_years, dtype: float64
UC patients Age at diagnosis quantile
0.25    31.0
0.50    37.0
0.75    43.0
Name: Age_at_diagnosis, dtype: float64
IBD patients Age quantile
0.25    38.25
0.50    44.50
0.75    56.25
Name: Age, dtype: float64
IBD patients BMI quantile
0.25    23.

In [7]:
## Anthropometric parameters
# CD patients endpoint
print(f'CD patients Age quantile\n{ust_skin_cd_40.Age.quantile([0.25,0.5,0.75])}')
print(f'CD patients BMI quantile\n{ust_skin_cd_40.BMI.quantile([0.25,0.5,0.75])}')
print(f'CD patients Disease duration years quantile\n{ust_skin_cd_40.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'CD patients Age at diagnosis quantile\n{ust_skin_cd_40.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')

# UC patients endpoint
print(f'UC patients Age quantile\n{ust_skin_uc_40.Age.quantile([0.25,0.5,0.75])}')
print(f'UC patients BMI quantile\n{ust_skin_uc_40.BMI.quantile([0.25,0.5,0.75])}')
print(f'UC patients Disease duration years quantile\n{ust_skin_uc_40.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'UC patients Age at diagnosis quantile\n{ust_skin_uc_40.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')

# IBD patients endpoint
print(f'IBD patients Age quantile\n{ust_skin_40.Age.quantile([0.25,0.5,0.75])}')
print(f'IBD patients BMI quantile\n{ust_skin_40.BMI.quantile([0.25,0.5,0.75])}')
print(f'IBD patients Disease duration years quantile\n{ust_skin_40.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'IBD patients Age at diagnosis quantile\n{ust_skin_40.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')

CD patients Age quantile
0.25    39.0
0.50    43.0
0.75    49.5
Name: Age, dtype: float64
CD patients BMI quantile
0.25    24.2325
0.50    26.2700
0.75    27.6750
Name: BMI, dtype: float64
CD patients Disease duration years quantile
0.25    10.25
0.50    12.50
0.75    16.50
Name: Disease_duration_years, dtype: float64
CD patients Age at diagnosis quantile
0.25    25.75
0.50    32.00
0.75    37.50
Name: Age_at_diagnosis, dtype: float64
UC patients Age quantile
0.25    63.0
0.50    63.0
0.75    63.0
Name: Age, dtype: float64
UC patients BMI quantile
0.25    25.66
0.50    25.66
0.75    25.66
Name: BMI, dtype: float64
UC patients Disease duration years quantile
0.25    15.0
0.50    15.0
0.75    15.0
Name: Disease_duration_years, dtype: float64
UC patients Age at diagnosis quantile
0.25    49.0
0.50    49.0
0.75    49.0
Name: Age_at_diagnosis, dtype: float64
IBD patients Age quantile
0.25    40.0
0.50    46.0
0.75    60.0
Name: Age, dtype: float64
IBD patients BMI quantile
0.25    25.26
0.5

In [8]:
## Clinical parameters
# CD baseline
ust_skin_cd_0['HBI'] = ust_skin_cd_0['HBI'].astype(float)
print(f'CD HBI\n{ust_skin_cd_0.HBI.quantile([0.25,0.5,0.75])}')
print(f'CD CRP\n{ust_skin_cd_0.CRP.quantile([0.25,0.5,0.75])}')
print(f'CD WBC\n{ust_skin_cd_0.WBC.quantile([0.25,0.5,0.75])}')
print(f'CD PLT\n{ust_skin_cd_0.PLT.quantile([0.25,0.5,0.75])}')
print(f'CD Ferritin\n{ust_skin_cd_0.Ferritin.quantile([0.25,0.5,0.75])}')
print(f'CD Hb\n{ust_skin_cd_0.Hb.quantile([0.25,0.5,0.75])}')
print(f'CD FC\n{ust_skin_cd_0.FC.quantile([0.25,0.5,0.75])}')
# UC baseline
ust_skin_uc_0['pMayo'] = ust_skin_uc_0['pMayo'].astype(float)
print(f'UC pMayo\n{ust_skin_uc_0.pMayo.quantile([0.25,0.5,0.75])}')
print(f'UC CRP\n{ust_skin_uc_0.CRP.quantile([0.25,0.5,0.75])}')
print(f'UC WBC\n{ust_skin_uc_0.WBC.quantile([0.25,0.5,0.75])}')
print(f'UC PLT\n{ust_skin_uc_0.PLT.quantile([0.25,0.5,0.75])}')
print(f'UC Ferritin\n{ust_skin_uc_0.Ferritin.quantile([0.25,0.5,0.75])}')
print(f'UC Hb\n{ust_skin_uc_0.Hb.quantile([0.25,0.5,0.75])}')
print(f'UC FC\n{ust_skin_uc_0.FC.quantile([0.25,0.5,0.75])}')
# IBD baseline
print(f'IBD CRP\n{ust_skin_0.CRP.quantile([0.25,0.5,0.75])}')
print(f'IBD WBC\n{ust_skin_0.WBC.quantile([0.25,0.5,0.75])}')
print(f'IBD PLT\n{ust_skin_0.PLT.quantile([0.25,0.5,0.75])}')
print(f'IBD Ferritin\n{ust_skin_0.Ferritin.quantile([0.25,0.5,0.75])}')
print(f'IBD Hb\n{ust_skin_0.Hb.quantile([0.25,0.5,0.75])}')
print(f'IBD FC\n{ust_skin_0.FC.quantile([0.25,0.5,0.75])}')

CD HBI
0.25    3.25
0.50    4.50
0.75    7.25
Name: HBI, dtype: float64
CD CRP
0.25    4.3
0.50    6.1
0.75    8.2
Name: CRP, dtype: float64
CD WBC
0.25    5.775
0.50    7.250
0.75    7.750
Name: WBC, dtype: float64
CD PLT
0.25    407.00
0.50    424.00
0.75    458.25
Name: PLT, dtype: float64
CD Ferritin
0.25    23.6425
0.50    45.5550
0.75    83.0750
Name: Ferritin, dtype: float64
CD Hb
0.25    128.00
0.50    134.50
0.75    147.75
Name: Hb, dtype: float64
CD FC
0.25     474.75
0.50    1071.00
0.75    1270.50
Name: FC, dtype: float64
UC pMayo
0.25    5.25
0.50    6.50
0.75    7.75
Name: pMayo, dtype: float64
UC CRP
0.25    3.1
0.50    5.5
0.75    7.9
Name: CRP, dtype: float64
UC WBC
0.25    14.525
0.50    15.450
0.75    16.375
Name: WBC, dtype: float64
UC PLT
0.25    376.0
0.50    378.0
0.75    380.0
Name: PLT, dtype: float64
UC Ferritin
0.25    16.2625
0.50    18.6850
0.75    21.1075
Name: Ferritin, dtype: float64
UC Hb
0.25    111.5
0.50    116.0
0.75    120.5
Name: Hb, dtype: float6

<ipython-input-8-4cb4dc10e238>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_skin_cd_0['HBI'] = ust_skin_cd_0['HBI'].astype(float)
<ipython-input-8-4cb4dc10e238>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_skin_uc_0['pMayo'] = ust_skin_uc_0['pMayo'].astype(float)


In [9]:
# CD endpoint
ust_skin_cd_40['HBI'] = ust_skin_cd_40['HBI'].astype(float)
print(f'CD HBI\n{ust_skin_cd_40.HBI.quantile([0.25,0.5,0.75])}')
print(f'CD CRP\n{ust_skin_cd_40.CRP.quantile([0.25,0.5,0.75])}')
print(f'CD WBC\n{ust_skin_cd_40.WBC.quantile([0.25,0.5,0.75])}')
print(f'CD PLT\n{ust_skin_cd_40.PLT.quantile([0.25,0.5,0.75])}')
print(f'CD Ferritin\n{ust_skin_cd_40.Ferritin.quantile([0.25,0.5,0.75])}')
print(f'CD Hb\n{ust_skin_cd_40.Hb.quantile([0.25,0.5,0.75])}')
print(f'CD FC\n{ust_skin_cd_40.FC.quantile([0.25,0.5,0.75])}')
# UC endpoint
ust_skin_uc_40['pMayo'] = ust_skin_uc_40['pMayo'].astype(float)
print(f'UC pMayo\n{ust_skin_uc_40.pMayo.quantile([0.25,0.5,0.75])}')
print(f'UC CRP\n{ust_skin_uc_40.CRP.quantile([0.25,0.5,0.75])}')
print(f'UC WBC\n{ust_skin_uc_40.WBC.quantile([0.25,0.5,0.75])}')
print(f'UC PLT\n{ust_skin_uc_40.PLT.quantile([0.25,0.5,0.75])}')
print(f'UC Ferritin\n{ust_skin_uc_40.Ferritin.quantile([0.25,0.5,0.75])}')
print(f'UC Hb\n{ust_skin_uc_40.Hb.quantile([0.25,0.5,0.75])}')
print(f'UC FC\n{ust_skin_uc_40.FC.quantile([0.25,0.5,0.75])}')
# IBD endpoint
print(f'IBD CRP\n{ust_skin_40.CRP.quantile([0.25,0.5,0.75])}')
print(f'IBD WBC\n{ust_skin_40.WBC.quantile([0.25,0.5,0.75])}')
print(f'IBD PLT\n{ust_skin_40.PLT.quantile([0.25,0.5,0.75])}')
print(f'IBD Ferritin\n{ust_skin_40.Ferritin.quantile([0.25,0.5,0.75])}')
print(f'IBD Hb\n{ust_skin_40.Hb.quantile([0.25,0.5,0.75])}')
print(f'IBD FC\n{ust_skin_40.FC.quantile([0.25,0.5,0.75])}')

CD HBI
0.25    2.5
0.50    3.0
0.75    3.5
Name: HBI, dtype: float64
CD CRP
0.25    2.225
0.50    3.150
0.75    4.250
Name: CRP, dtype: float64
CD WBC
0.25    6.175
0.50    6.750
0.75    7.000
Name: WBC, dtype: float64
CD PLT
0.25    292.25
0.50    322.00
0.75    409.00
Name: PLT, dtype: float64
CD Ferritin
0.25     5.2575
0.50    10.4900
0.75    21.3350
Name: Ferritin, dtype: float64
CD Hb
0.25    124.25
0.50    128.50
0.75    138.00
Name: Hb, dtype: float64
CD FC
0.25    364.75
0.50    498.50
0.75    889.50
Name: FC, dtype: float64
UC pMayo
0.25    1.0
0.50    1.0
0.75    1.0
Name: pMayo, dtype: float64
UC CRP
0.25    2.3
0.50    2.3
0.75    2.3
Name: CRP, dtype: float64
UC WBC
0.25    7.5
0.50    7.5
0.75    7.5
Name: WBC, dtype: float64
UC PLT
0.25    179.0
0.50    179.0
0.75    179.0
Name: PLT, dtype: float64
UC Ferritin
0.25    36.39
0.50    36.39
0.75    36.39
Name: Ferritin, dtype: float64
UC Hb
0.25    156.0
0.50    156.0
0.75    156.0
Name: Hb, dtype: float64
UC FC
0.25    28

<ipython-input-9-ffc8b6e458bb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_skin_cd_40['HBI'] = ust_skin_cd_40['HBI'].astype(float)
<ipython-input-9-ffc8b6e458bb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_skin_uc_40['pMayo'] = ust_skin_uc_40['pMayo'].astype(float)


In [10]:
# HC baseline
print(f'HC Age\n{ust_skin_hc_0.Age.quantile([0.25,0.5,0.75])}')
print(f'HC BMI\n{ust_skin_hc_0.BMI.quantile([0.25,0.5,0.75])}')
print(f'HC CRP\n{ust_skin_hc_0.CRP.quantile([0.25,0.5,0.75])}')
print(f'HC WBC\n{ust_skin_hc_0.WBC.quantile([0.25,0.5,0.75])}')
print(f'HC PLT\n{ust_skin_hc_0.PLT.quantile([0.25,0.5,0.75])}')
print(f'HC Ferritin\n{ust_skin_hc_0.Ferritin.quantile([0.25,0.5,0.75])}')
print(f'HC Hb\n{ust_skin_hc_0.Hb.quantile([0.25,0.5,0.75])}')
print(f'HC FC\n{ust_skin_hc_0.FC.quantile([0.25,0.5,0.75])}')

HC Age
0.25    29.0
0.50    34.5
0.75    40.0
Name: Age, dtype: float64
HC BMI
0.25    21.5625
0.50    23.7600
0.75    26.9150
Name: BMI, dtype: float64
HC CRP
0.25    0.400
0.50    0.700
0.75    1.775
Name: CRP, dtype: float64
HC WBC
0.25    4.825
0.50    5.800
0.75    6.825
Name: WBC, dtype: float64
HC PLT
0.25    239.25
0.50    250.50
0.75    283.50
Name: PLT, dtype: float64
HC Ferritin
0.25   NaN
0.50   NaN
0.75   NaN
Name: Ferritin, dtype: float64
HC Hb
0.25    135.25
0.50    141.00
0.75    151.50
Name: Hb, dtype: float64
HC FC
0.25   NaN
0.50   NaN
0.75   NaN
Name: FC, dtype: float64


In [11]:
# modify for pairwise tests
ust_skin_cd_0w = ust_skin_cd_0.drop(["P18_SJ75_I_D", "P43_BR63_I_D"])
ust_skin_0w = ust_skin_0.drop(["P18_SJ75_I_D", "P11_GT85_I_D", "P43_BR63_I_D"])
ust_skin_uc_0w = ust_skin_uc_0.drop("P11_GT85_I_D")

# calculate pairwise comparison between week 0 and week 40 in CD patients
print(f'{stats.wilcoxon(ust_skin_cd_0w["HBI"], ust_skin_cd_40["HBI"])}')
print(f'CRP {stats.wilcoxon(ust_skin_cd_0w["CRP"], ust_skin_cd_40["CRP"])}')
print(f'WBC {stats.wilcoxon(ust_skin_cd_0w["WBC"], ust_skin_cd_40["WBC"])}')
print(f'PLT {stats.wilcoxon(ust_skin_cd_0w["PLT"], ust_skin_cd_40["PLT"])}')
print(f'Ferritin {stats.wilcoxon(ust_skin_cd_0w["Ferritin"], ust_skin_cd_40["Ferritin"])}')
print(f'Hb {stats.wilcoxon(ust_skin_cd_0w["Hb"], ust_skin_cd_40["Hb"])}')
print(f'FC {stats.wilcoxon(ust_skin_cd_0w["FC"], ust_skin_cd_40["FC"])}')

# correct for multiple hypothesis testing
wilcox_pval = []
wilcox_pval.append(stats.wilcoxon(ust_skin_cd_0w["HBI"], ust_skin_cd_40["HBI"])[1])
wilcox_pval.append(stats.wilcoxon(ust_skin_cd_0w["CRP"], ust_skin_cd_40["CRP"])[1])
wilcox_pval.append(stats.wilcoxon(ust_skin_cd_0w["WBC"], ust_skin_cd_40["WBC"])[1])
wilcox_pval.append(stats.wilcoxon(ust_skin_cd_0w["PLT"], ust_skin_cd_40["PLT"])[1])
wilcox_pval.append(stats.wilcoxon(ust_skin_cd_0w["Ferritin"], ust_skin_cd_40["Ferritin"])[1])
wilcox_pval.append(stats.wilcoxon(ust_skin_cd_0w["Hb"], ust_skin_cd_40["Hb"])[1])
wilcox_pval.append(stats.wilcoxon(ust_skin_cd_0w["FC"], ust_skin_cd_40["FC"])[1])
print(f'Bonferroni corrected pvals CRP, WBC, PLT, Ferritin, Hb, FC = {sm.multitest.multipletests(wilcox_pval, alpha=0.05, method="bonferroni")[1]}')

WilcoxonResult(statistic=1.0, pvalue=0.27630291733748347)
CRP WilcoxonResult(statistic=1.0, pvalue=0.25)
WBC WilcoxonResult(statistic=1.0, pvalue=0.25)
PLT WilcoxonResult(statistic=2.0, pvalue=0.375)
Ferritin WilcoxonResult(statistic=1.0, pvalue=0.25)
Hb WilcoxonResult(statistic=3.5, pvalue=0.625)
FC WilcoxonResult(statistic=3.0, pvalue=0.625)
Bonferroni corrected pvals CRP, WBC, PLT, Ferritin, Hb, FC = [1. 1. 1. 1. 1. 1. 1.]


/Users/dagmarschierova/MEGA/MBÚ/venv_MBU/lib/python3.9/site-packages/scipy/stats/morestats.py:2957: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/dagmarschierova/MEGA/MBÚ/venv_MBU/lib/python3.9/site-packages/scipy/stats/morestats.py:2971: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [12]:
# calculate comparison between HC and CD
print(f'CRP {stats.mannwhitneyu(ust_skin_cd_0["CRP"], ust_skin_hc_0["CRP"])}')
print(f'WBC {stats.mannwhitneyu(ust_skin_cd_0["WBC"], ust_skin_hc_0["WBC"])}')
print(f'PLT {stats.mannwhitneyu(ust_skin_cd_0["PLT"], ust_skin_hc_0["PLT"])}')
print(f'Hb {stats.mannwhitneyu(ust_skin_cd_0["Hb"], ust_skin_hc_0["Hb"])}')
mannwu_pval = []
mannwu_pval.append(stats.mannwhitneyu(ust_skin_cd_0["CRP"], ust_skin_hc_0["CRP"])[1])
mannwu_pval.append(stats.mannwhitneyu(ust_skin_cd_0["WBC"], ust_skin_hc_0["WBC"])[1])
mannwu_pval.append(stats.mannwhitneyu(ust_skin_cd_0["PLT"], ust_skin_hc_0["PLT"])[1])
mannwu_pval.append(stats.mannwhitneyu(ust_skin_cd_0["Hb"], ust_skin_hc_0["Hb"])[1])
print(f'Bonferroni corrected pvals CRP, WBC, PLT, Hb = {sm.multitest.multipletests(mannwu_pval, alpha=0.05, method="bonferroni")[1]}')

CRP MannwhitneyuResult(statistic=5.0, pvalue=0.00034575211202118454)
WBC MannwhitneyuResult(statistic=42.0, pvalue=0.09385718542193427)
PLT MannwhitneyuResult(statistic=2.0, pvalue=0.00018841243889059474)
Hb MannwhitneyuResult(statistic=43.0, pvalue=0.10350465355058824)
Bonferroni corrected pvals CRP, WBC, PLT, Hb = [0.00138301 0.37542874 0.00075365 0.41401861]


In [13]:
# calculate pairwise comparison between week 0 and week 40 in IBD patients
print(f'CRP {stats.wilcoxon(ust_skin_0w["CRP"], ust_skin_40["CRP"])}')
print(f'WBC {stats.wilcoxon(ust_skin_0w["WBC"], ust_skin_40["WBC"])}')
print(f'PLT {stats.wilcoxon(ust_skin_0w["PLT"], ust_skin_40["PLT"])}')
print(f'Ferritin {stats.wilcoxon(ust_skin_0w["Ferritin"], ust_skin_40["Ferritin"])}')
print(f'Hb {stats.wilcoxon(ust_skin_0w["Hb"], ust_skin_40["Hb"])}')
print(f'FC {stats.wilcoxon(ust_skin_0w["FC"], ust_skin_40["FC"])}')
wilcox_pval = []
wilcox_pval.append(stats.wilcoxon(ust_skin_cd_0w["HBI"], ust_skin_cd_40["HBI"])[1])
wilcox_pval.append(stats.wilcoxon(ust_skin_uc_0w["pMayo"], ust_skin_uc_40["pMayo"])[1])
wilcox_pval.append(stats.wilcoxon(ust_skin_0w["CRP"], ust_skin_40["CRP"])[1])
wilcox_pval.append(stats.wilcoxon(ust_skin_0w["WBC"], ust_skin_40["WBC"])[1])
wilcox_pval.append(stats.wilcoxon(ust_skin_0w["PLT"], ust_skin_40["PLT"])[1])
wilcox_pval.append(stats.wilcoxon(ust_skin_0w["Ferritin"], ust_skin_40["Ferritin"])[1])
wilcox_pval.append(stats.wilcoxon(ust_skin_0w["Hb"], ust_skin_40["Hb"])[1])
wilcox_pval.append(stats.wilcoxon(ust_skin_0w["FC"], ust_skin_40["FC"])[1])
print(f'Bonferroni corrected pvals CRP, WBC, PLT, Ferritin, Hb, FC = {sm.multitest.multipletests(wilcox_pval, alpha=0.05, method="bonferroni")[1]}')

CRP WilcoxonResult(statistic=1.0, pvalue=0.125)
WBC WilcoxonResult(statistic=1.0, pvalue=0.125)
PLT WilcoxonResult(statistic=2.0, pvalue=0.1875)
Ferritin WilcoxonResult(statistic=6.0, pvalue=0.8125)
Hb WilcoxonResult(statistic=3.5, pvalue=0.3125)
FC WilcoxonResult(statistic=3.0, pvalue=0.3125)
Bonferroni corrected pvals CRP, WBC, PLT, Ferritin, Hb, FC = [1. 1. 1. 1. 1. 1. 1. 1.]


In [14]:
# calculate comparison between HC and IBD
print(f'CRP {stats.mannwhitneyu(ust_skin_0["CRP"], ust_skin_hc_0["CRP"])}')
print(f'WBC {stats.mannwhitneyu(ust_skin_0["WBC"], ust_skin_hc_0["WBC"])}')
print(f'PLT {stats.mannwhitneyu(ust_skin_0["PLT"], ust_skin_hc_0["PLT"])}')
print(f'Hb {stats.mannwhitneyu(ust_skin_0["Hb"], ust_skin_hc_0["Hb"])}')
mannwu_pval = []
mannwu_pval.append(stats.mannwhitneyu(ust_skin_cd_0["CRP"], ust_skin_hc_0["CRP"])[1])
mannwu_pval.append(stats.mannwhitneyu(ust_skin_cd_0["WBC"], ust_skin_hc_0["WBC"])[1])
mannwu_pval.append(stats.mannwhitneyu(ust_skin_cd_0["PLT"], ust_skin_hc_0["PLT"])[1])
mannwu_pval.append(stats.mannwhitneyu(ust_skin_cd_0["Hb"], ust_skin_hc_0["Hb"])[1])
print(f'Bonferroni corrected pvals CRP, WBC, PLT, Hb = {sm.multitest.multipletests(mannwu_pval, alpha=0.05, method="bonferroni")[1]}')

CRP MannwhitneyuResult(statistic=16.0, pvalue=0.00039299050457536073)
WBC MannwhitneyuResult(statistic=42.0, pvalue=0.01633994665975371)
PLT MannwhitneyuResult(statistic=2.0, pvalue=3.0331488322273756e-05)
Hb MannwhitneyuResult(statistic=43.0, pvalue=0.018320218431394194)
Bonferroni corrected pvals CRP, WBC, PLT, Hb = [0.00138301 0.37542874 0.00075365 0.41401861]


# Stool microbiome

In [15]:
# load metadata stool
metadata_stool = pd.read_csv("/Users/dagmarschierova/MEGA/MBÚ/USTEK/NGS/input/metadata_ustek_new.tsv", sep='\t')
metadata_stool = metadata_stool.iloc[1:]
metadata_stool.set_index('Label', inplace=True)
metadata_stool.drop(labels=["K26_1", "K28_1"], axis=0, inplace=True)

# convert numeric columns to float
metadata_stool['Age'] = metadata_stool['Age'].astype(float)
metadata_stool['BMI'] = metadata_stool['BMI'].astype(float)
metadata_stool['CRP'] = metadata_stool['CRP'].astype(float)
metadata_stool['WBC'] = metadata_stool['WBC'].astype(float)
metadata_stool['PLT'] = metadata_stool['PLT'].astype(float)
metadata_stool['Ferritin'] = metadata_stool['Ferritin'].astype(float)
metadata_stool['Hb'] = metadata_stool['Hb'].astype(float)
metadata_stool['FC'] = metadata_stool['FC'].astype(float)

# filter patients on UST
ust_stool = metadata_stool[metadata_stool["Drug"]=="UST"]

#convert numeric columns to float
ust_stool['Disease_duration_years'] = ust_stool['Disease_duration_years'].astype(float)
ust_stool['Age_at_diagnosis'] = ust_stool['Age_at_diagnosis'].astype(float)

# filter out HC
ust_stool_hc = metadata_stool[metadata_stool["Dg"]=="HC"]

# load HC metadata
metadata_hc = pd.read_csv("/Users/dagmarschierova/MEGA/MBÚ/TNF/clinical_param/Healthy_controls.tsv", sep='\t')
metadata_hc.set_index('SampleID', inplace=True)

# merge HC metadata with biomarker values
ust_stool_hc = ust_stool_hc.merge(metadata_hc, how='left', left_index=True, right_index=True)
ust_stool_hc.drop(columns=['Age_x', 'Sex_x', 'BMI_x', 'CRP_x', 'WBC_x', 'PLT_x', 'Hb_x'], inplace=True)
ust_stool_hc.rename(columns={"Age_y": "Age", "Sex_y": "Sex", "BMI_y": "BMI", 'CRP_y': 'CRP', 'WBC_y': 'WBC', 'PLT_y': 'PLT', 'Hb_y': "Hb"}, inplace=True)


<ipython-input-15-775d6c922e22>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_stool['Disease_duration_years'] = ust_stool['Disease_duration_years'].astype(float)
<ipython-input-15-775d6c922e22>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_stool['Age_at_diagnosis'] = ust_stool['Age_at_diagnosis'].astype(float)


In [16]:
# baseline
ust_stool_cd_0 = ust_stool[(ust_stool["Dg"]=="CD") & (ust_stool["Week"]=="week_0")]
ust_stool_uc_0 = ust_stool[(ust_stool["Dg"]=="UC") & (ust_stool["Week"]=="week_0")]
ust_stool_0 = ust_stool[ust_stool["Week"]=="week_0"]

# endpoint
ust_stool_cd_40 = ust_stool[(ust_stool["Dg"]=="CD") & (ust_stool["Week"]=="week_40")]
ust_stool_uc_40 = ust_stool[(ust_stool["Dg"]=="UC") & (ust_stool["Week"]=="week_40")]
ust_stool_40 = ust_stool[ust_stool["Week"]=="week_40"]

# all patient one visit
ust_stool_0all = ust_stool_0.append(ust_stool.loc["30_KM_2"])

# healthy controls
ust_stool_hc_0 = ust_stool_hc[ust_stool_hc["Week"]=="week_0"]

In [17]:
print("Number of IBD patients at baseline (week 0) = ", ust_stool[(ust_stool['Disease_status']=="IBD") & (ust_stool['Week']=="week_0")]["Patient"].nunique())
print("Number of IBD patients at endpoint (week 40) = ", ust_stool[(ust_stool['Disease_status']=="IBD") & (ust_stool['Week']=="week_40")]["Patient"].nunique())
print("Number of IBD females at baseline (week 0) = ", ust_stool[(ust_stool['Sex']=="F") & (ust_stool['Week']=="week_0")]["Patient"].nunique())
print("Number of IBD males at baseline (week 0) = ", ust_stool[(ust_stool['Sex']=="M") & (ust_stool['Week']=="week_0")]["Patient"].nunique())
print("Number of IBD females at endpoint (week 40) = ", ust_stool[(ust_stool['Sex']=="F") & (ust_stool['Week']=="week_40")]["Patient"].nunique())
print("Number of IBD males at endpoint (week 40) = ", ust_stool[(ust_stool['Sex']=="M") & (ust_stool['Week']=="week_40")]["Patient"].nunique())
print("Number of CD patients =", ust_stool[(ust_stool['Dg']=="CD")]["Patient"].nunique())
print("Number of UC patients =", ust_stool[(ust_stool['Dg']=="UC")]["Patient"].nunique())
print("Number of CD patients at baseline (week 0) =", ust_stool[(ust_stool['Dg']=="CD") & (ust_stool['Week']=="week_0")]["Patient"].nunique())
print("Number of UC patients at baseline (week 0) =", ust_stool[(ust_stool['Dg']=="UC") & (ust_stool['Week']=="week_0")]["Patient"].nunique())
print("Number of CD patients at baseline (week 40) =", ust_stool[(ust_stool['Dg']=="CD") & (ust_stool['Week']=="week_40")]["Patient"].nunique())
print("Number of UC patients at baseline (week 40) =", ust_stool[(ust_stool['Dg']=="UC") & (ust_stool['Week']=="week_40")]["Patient"].nunique())
print("Number of remitters at baseline (week 0) = ", ust_stool[(ust_stool['Disease_severity_scale4']=="remission") & (ust_stool['Week']=="week_0")]["Patient"].nunique())
print("Number of remitters at endpoint (week 40) = ", ust_stool[(ust_stool['Disease_severity_scale4']=="remission") & (ust_stool['Week']=="week_40")]["Patient"].nunique())
print("Number of HC = ", ust_stool_hc[ust_stool_hc['Disease_status']=="HC"]["Patient"].nunique())
print("Number of HC females entering the study = ", ust_stool_hc[(ust_stool_hc['Sex']=="F")]["Patient"].nunique())
print("Number of HC males entering the study = ", ust_stool_hc[(ust_stool_hc['Sex']=="M")]["Patient"].nunique())

Number of IBD patients at baseline (week 0) =  10
Number of IBD patients at endpoint (week 40) =  8
Number of IBD females at baseline (week 0) =  4
Number of IBD males at baseline (week 0) =  6
Number of IBD females at endpoint (week 40) =  4
Number of IBD males at endpoint (week 40) =  4
Number of CD patients = 9
Number of UC patients = 2
Number of CD patients at baseline (week 0) = 8
Number of UC patients at baseline (week 0) = 2
Number of CD patients at baseline (week 40) = 6
Number of UC patients at baseline (week 40) = 2
Number of remitters at baseline (week 0) =  5
Number of remitters at endpoint (week 40) =  5
Number of HC =  39
Number of HC females entering the study =  0
Number of HC males entering the study =  0


In [18]:
# Anthropometric data
# IBD all patients visit
print(f'IBD Age\n{ust_stool_0all.Age.quantile([0.25,0.5,0.75])}')
print(f'IBD BMI\n{ust_stool_0all.BMI.quantile([0.25,0.5,0.75])}')
print(f'IBD Disease duration years\n{ust_stool_0all.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'IBD Age at diagnosis\n{ust_stool_0all.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')

# IBD baseline
print(f'IBD Age\n{ust_stool_0.Age.quantile([0.25,0.5,0.75])}')
print(f'IBD BMI\n{ust_stool_0.BMI.quantile([0.25,0.5,0.75])}')
print(f'IBD Disease duration years\n{ust_stool_0.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'IBD Age at diagnosis\n{ust_stool_0.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')

# IBD endpoint
print(f'IBD Age\n{ust_stool_40.Age.quantile([0.25,0.5,0.75])}')
print(f'IBD BMI\n{ust_stool_40.BMI.quantile([0.25,0.5,0.75])}')
print(f'IBD Disease duration years\n{ust_stool_40.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'IBD Age at diagnosis\n{ust_stool_40.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')

IBD Age
0.25    34.0
0.50    40.0
0.75    50.5
Name: Age, dtype: float64
IBD BMI
0.25    21.90
0.50    25.40
0.75    27.35
Name: BMI, dtype: float64
IBD Disease duration years
0.25     6.5
0.50    10.0
0.75    13.0
Name: Disease_duration_years, dtype: float64
IBD Age at diagnosis
0.25    26.0
0.50    32.0
0.75    41.0
Name: Age_at_diagnosis, dtype: float64
IBD Age
0.25    33.5
0.50    40.0
0.75    52.0
Name: Age, dtype: float64
IBD BMI
0.25    20.750
0.50    25.050
0.75    27.225
Name: BMI, dtype: float64
IBD Disease duration years
0.25     6.25
0.50    10.50
0.75    13.50
Name: Disease_duration_years, dtype: float64
IBD Age at diagnosis
0.25    25.5
0.50    31.0
0.75    41.5
Name: Age_at_diagnosis, dtype: float64
IBD Age
0.25    35.75
0.50    40.00
0.75    49.50
Name: Age, dtype: float64
IBD BMI
0.25    23.80
0.50    25.50
0.75    26.25
Name: BMI, dtype: float64
IBD Disease duration years
0.25     9.50
0.50    10.50
0.75    14.25
Name: Disease_duration_years, dtype: float64
IBD Age at

In [19]:
# CD baseline
ust_stool_cd_0['HBI'] = ust_stool_cd_0['HBI'].astype(float)
print(f'CD HBI\n{ust_stool_cd_0.HBI.quantile([0.25,0.5,0.75])}')

# UC baseline
ust_stool_uc_0['pMayo'] = ust_stool_uc_0['pMayo'].astype(float)
print(f'UC pMayo\n{ust_stool_uc_0.pMayo.quantile([0.25,0.5,0.75])}')

# IBD baseline
print(f'IBD CRP\n{ust_stool_0.CRP.quantile([0.25,0.5,0.75])}')
print(f'IBD WBC\n{ust_stool_0.WBC.quantile([0.25,0.5,0.75])}')
print(f'IBD PLT\n{ust_stool_0.PLT.quantile([0.25,0.5,0.75])}')
print(f'IBD Ferritin\n{ust_stool_0.Ferritin.quantile([0.25,0.5,0.75])}')
print(f'IBD Hb\n{ust_stool_0.Hb.quantile([0.25,0.5,0.75])}')
print(f'IBD FC\n{ust_stool_0.FC.quantile([0.25,0.5,0.75])}')

CD HBI
0.25    2.5
0.50    4.0
0.75    5.5
Name: HBI, dtype: float64
UC pMayo
0.25    5.25
0.50    6.50
0.75    7.75
Name: pMayo, dtype: float64
IBD CRP
0.25    4.300
0.50    7.250
0.75    9.925
Name: CRP, dtype: float64
IBD WBC
0.25    5.950
0.50    8.150
0.75    9.775
Name: WBC, dtype: float64
IBD PLT
0.25    316.0
0.50    378.0
0.75    420.0
Name: PLT, dtype: float64
IBD Ferritin
0.25    18.2275
0.50    28.3450
0.75    83.0750
Name: Ferritin, dtype: float64
IBD Hb
0.25    125.25
0.50    132.50
0.75    147.75
Name: Hb, dtype: float64
IBD FC
0.25     346.25
0.50    1071.00
0.75    1389.00
Name: FC, dtype: float64


<ipython-input-19-a3dbae18a28a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_stool_cd_0['HBI'] = ust_stool_cd_0['HBI'].astype(float)
<ipython-input-19-a3dbae18a28a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_stool_uc_0['pMayo'] = ust_stool_uc_0['pMayo'].astype(float)


In [20]:
# CD endpoint
ust_stool_cd_40['HBI'] = ust_stool_cd_40['HBI'].astype(float)
print(f'CD HBI\n{ust_stool_cd_40.HBI.quantile([0.25,0.5,0.75])}')

# UC endpoint
ust_stool_uc_40['pMayo'] = ust_stool_uc_40['pMayo'].astype(float)
print(f'UC pMayo\n{ust_stool_uc_40.pMayo.quantile([0.25,0.5,0.75])}')

# IBD endpoint
print(f'IBD CRP\n{ust_stool_40.CRP.quantile([0.25,0.5,0.75])}')
print(f'IBD WBC\n{ust_stool_40.WBC.quantile([0.25,0.5,0.75])}')
print(f'IBD PLT\n{ust_stool_40.PLT.quantile([0.25,0.5,0.75])}')
print(f'IBD Ferritin\n{ust_stool_40.Ferritin.quantile([0.25,0.5,0.75])}')
print(f'IBD Hb\n{ust_stool_40.Hb.quantile([0.25,0.5,0.75])}')
print(f'IBD FC\n{ust_stool_40.FC.quantile([0.25,0.5,0.75])}')

CD HBI
0.25    1.5
0.50    3.0
0.75    4.5
Name: HBI, dtype: float64
UC pMayo
0.25    1.25
0.50    1.50
0.75    1.75
Name: pMayo, dtype: float64
IBD CRP
0.25    2.225
0.50    2.650
0.75    4.075
Name: CRP, dtype: float64
IBD WBC
0.25    6.875
0.50    7.250
0.75    9.525
Name: WBC, dtype: float64
IBD PLT
0.25    224.00
0.50    290.50
0.75    347.75
Name: PLT, dtype: float64
IBD Ferritin
0.25     7.085
0.50    14.375
0.75    37.655
Name: Ferritin, dtype: float64
IBD Hb
0.25    121.25
0.50    128.50
0.75    153.75
Name: Hb, dtype: float64
IBD FC
0.25     61.00
0.50    177.00
0.75    483.75
Name: FC, dtype: float64


<ipython-input-20-57291f38893f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_stool_cd_40['HBI'] = ust_stool_cd_40['HBI'].astype(float)
<ipython-input-20-57291f38893f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_stool_uc_40['pMayo'] = ust_stool_uc_40['pMayo'].astype(float)


In [21]:
# HC baseline
print(f'HC Age\n{ust_stool_hc_0.Age.quantile([0.25,0.5,0.75])}')
print(f'HC BMI\n{ust_stool_hc_0.BMI.quantile([0.25,0.5,0.75])}')
print(f'HC CRP\n{ust_stool_hc_0.CRP.quantile([0.25,0.5,0.75])}')
print(f'HC WBC\n{ust_stool_hc_0.WBC.quantile([0.25,0.5,0.75])}')
print(f'HC PLT\n{ust_stool_hc_0.PLT.quantile([0.25,0.5,0.75])}')
print(f'HC Hb\n{ust_stool_hc_0.Hb.quantile([0.25,0.5,0.75])}')

HC Age
0.25   NaN
0.50   NaN
0.75   NaN
Name: Age, dtype: float64
HC BMI
0.25   NaN
0.50   NaN
0.75   NaN
Name: BMI, dtype: float64
HC CRP
0.25   NaN
0.50   NaN
0.75   NaN
Name: CRP, dtype: float64
HC WBC
0.25   NaN
0.50   NaN
0.75   NaN
Name: WBC, dtype: float64
HC PLT
0.25   NaN
0.50   NaN
0.75   NaN
Name: PLT, dtype: float64
HC Hb
0.25   NaN
0.50   NaN
0.75   NaN
Name: Hb, dtype: float64


In [22]:
# modify for pairwise tests
ust_stool_0w = ust_stool_0.drop(["18_SJ_1", "35_AJ_1", "43_BR_1"])
ust_stool_40w = ust_stool_40.drop(["30_KM_6"])
ust_stool_cd_0w = ust_stool_cd_0.drop(["18_SJ_1", "35_AJ_1", "43_BR_1"])
ust_stool_cd_40w = ust_stool_cd_40.drop(["30_KM_6"])

# calculate pairwise comparison between week 0 and week 40 in IBD patients
print(f'HBI {stats.wilcoxon(ust_stool_cd_0w["HBI"], ust_stool_cd_40w["HBI"])}')
print(f'pMayo {stats.wilcoxon(ust_stool_uc_0["pMayo"], ust_stool_uc_40["pMayo"])}')
print(f'CRP {stats.wilcoxon(ust_stool_0w["CRP"], ust_stool_40w["CRP"])}')
print(f'WBC {stats.wilcoxon(ust_stool_0w["WBC"], ust_stool_40w["WBC"])}')
print(f'PLT {stats.wilcoxon(ust_stool_0w["PLT"], ust_stool_40w["PLT"])}')
print(f'Ferritin {stats.wilcoxon(ust_stool_0w["Ferritin"], ust_stool_40w["Ferritin"])}')
print(f'Hb {stats.wilcoxon(ust_stool_0w["Hb"], ust_stool_40w["Hb"])}')
print(f'FC {stats.wilcoxon(ust_stool_0w["FC"], ust_stool_40w["FC"])}')

# correct for multiple hypothesis testing
wilcox_pval = []
wilcox_pval.append(stats.wilcoxon(ust_stool_cd_0w["HBI"], ust_stool_cd_40w["HBI"])[1])
wilcox_pval.append(stats.wilcoxon(ust_stool_uc_0["pMayo"], ust_stool_uc_40["pMayo"])[1])
wilcox_pval.append(stats.wilcoxon(ust_stool_0w["CRP"], ust_stool_40w["CRP"])[1])
wilcox_pval.append(stats.wilcoxon(ust_stool_0w["WBC"], ust_stool_40w["WBC"])[1])
wilcox_pval.append(stats.wilcoxon(ust_stool_0w["PLT"], ust_stool_40w["PLT"])[1])
wilcox_pval.append(stats.wilcoxon(ust_stool_0w["Ferritin"], ust_stool_40w["Ferritin"])[1])
wilcox_pval.append(stats.wilcoxon(ust_stool_0w["Hb"], ust_stool_40w["Hb"])[1])
wilcox_pval.append(stats.wilcoxon(ust_stool_0w["FC"], ust_stool_40w["FC"])[1])
print(f'Bonferroni corrected pvals CRP, WBC, PLT, Ferritin, Hb, FC = {sm.multitest.multipletests(wilcox_pval, alpha=0.05, method="bonferroni")[1]}')

HBI WilcoxonResult(statistic=4.5, pvalue=0.8539232992870668)
pMayo WilcoxonResult(statistic=0.0, pvalue=0.5)
CRP WilcoxonResult(statistic=3.0, pvalue=0.078125)
WBC WilcoxonResult(statistic=4.0, pvalue=0.109375)
PLT WilcoxonResult(statistic=10.0, pvalue=0.578125)
Ferritin WilcoxonResult(statistic=9.0, pvalue=0.46875)
Hb WilcoxonResult(statistic=13.5, pvalue=0.9375)
FC WilcoxonResult(statistic=6.0, pvalue=0.21875)
Bonferroni corrected pvals CRP, WBC, PLT, Ferritin, Hb, FC = [1.    1.    0.625 0.875 1.    1.    1.    1.   ]


In [23]:
# calculate comparison between HC and IBD
print(f'CRP {stats.mannwhitneyu(ust_stool_0["CRP"], ust_stool_hc_0["CRP"])}')
print(f'WBC {stats.mannwhitneyu(ust_stool_0["WBC"], ust_stool_hc_0["WBC"])}')
print(f'PLT {stats.mannwhitneyu(ust_stool_0["PLT"], ust_stool_hc_0["PLT"])}')
print(f'Hb {stats.mannwhitneyu(ust_stool_0["Hb"], ust_stool_hc_0["Hb"])}')

# correct for multiple hypothesis testing
mannwu_pval = []
mannwu_pval.append(stats.mannwhitneyu(ust_stool_0["CRP"], ust_stool_hc_0["CRP"])[1])
mannwu_pval.append(stats.mannwhitneyu(ust_stool_0["WBC"], ust_stool_hc_0["WBC"])[1])
mannwu_pval.append(stats.mannwhitneyu(ust_stool_0["PLT"], ust_stool_hc_0["PLT"])[1])
mannwu_pval.append(stats.mannwhitneyu(ust_stool_0["Hb"], ust_stool_hc_0["Hb"])[1])
print(f'Bonferroni corrected pvals CRP, WBC, PLT, Hb = {sm.multitest.multipletests(mannwu_pval, alpha=0.05, method="bonferroni")[1]}')

CRP MannwhitneyuResult(statistic=0.0, pvalue=7.001878420978284e-07)
WBC MannwhitneyuResult(statistic=0.0, pvalue=6.997555439203418e-07)
PLT MannwhitneyuResult(statistic=0.0, pvalue=7.001878420978284e-07)
Hb MannwhitneyuResult(statistic=0.0, pvalue=7.001878420978284e-07)
Bonferroni corrected pvals CRP, WBC, PLT, Hb = [2.80075137e-06 2.79902218e-06 2.80075137e-06 2.80075137e-06]


## Biomarkers

In [24]:
# load biomarkers patients
metadata_biomarkers = pd.read_csv("/Users/dagmarschierova/MEGA/MBÚ/USTEK/statistics/vysledky_biomarkery_pacientiUSTekinumab_20.11.25.tsv", sep='\t')
metadata_biomarkers.set_index('SampleID', inplace=True)

# load biomarkers healthy controls
metadata_biomarkers_hc = pd.read_csv("/Users/dagmarschierova/MEGA/MBÚ/USTEK/statistics/vysledky_biomarkery_kontroly_20.11.25.tsv", sep='\t')
metadata_biomarkers_hc.set_index('SampleID', inplace=True)

# merge metadata with biomarkers patients
metadata_biomarkers = metadata_biomarkers.merge(metadata_stool, how='left', left_index=True, right_index=True)
metadata_biomarkers.drop(columns=['Patient_y', 'Week_y'], inplace=True)
metadata_biomarkers.rename(columns={"Patient_x": "Patient", "Week_x": "Week"}, inplace=True)

# merge metadata with biomarkers healthy controls
metadata_biomarkers_hc = metadata_biomarkers_hc.merge(metadata_hc, how='left', left_index=True, right_index=True)

# add values for patient 30_KM_I, 18_SJ_VI, 2_ZM
metadata_biomarkers.at['2_ZM_1', 'Disease_status'] = "IBD"
metadata_biomarkers.at['2_ZM_2', 'Disease_status'] = "IBD"
metadata_biomarkers.at['2_ZM_3', 'Disease_status'] = "IBD"
metadata_biomarkers.at['2_ZM_4', 'Disease_status'] = "IBD"
metadata_biomarkers.at['2_ZM_5', 'Disease_status'] = "IBD"
metadata_biomarkers.at['2_ZM_6', 'Disease_status'] = "IBD"
metadata_biomarkers.at['2_ZM_1', 'Dg'] = metadata_skin.loc["P2_ZM87_I_D", "Dg"]
metadata_biomarkers.at['2_ZM_2', 'Dg'] = metadata_skin.loc["P2_ZM87_II_D", "Dg"]
metadata_biomarkers.at['2_ZM_3', 'Dg'] = metadata_skin.loc["P2_ZM87_III_D", "Dg"]
metadata_biomarkers.at['2_ZM_4', 'Dg'] = metadata_skin.loc["P2_ZM87_IV_D", "Dg"]
metadata_biomarkers.at['2_ZM_5', 'Dg'] = metadata_skin.loc["P2_ZM87_V_D", "Dg"]
metadata_biomarkers.at['2_ZM_6', 'Dg'] = metadata_skin.loc["P2_ZM87_VI_D", "Dg"]
metadata_biomarkers.at['2_ZM_1', 'Sex'] = metadata_skin.loc["P2_ZM87_I_D", "Sex"]
metadata_biomarkers.at['2_ZM_2', 'Sex'] = metadata_skin.loc["P2_ZM87_II_D", "Sex"]
metadata_biomarkers.at['2_ZM_3', 'Sex'] = metadata_skin.loc["P2_ZM87_III_D", "Sex"]
metadata_biomarkers.at['2_ZM_4', 'Sex'] = metadata_skin.loc["P2_ZM87_IV_D", "Sex"]
metadata_biomarkers.at['2_ZM_5', 'Sex'] = metadata_skin.loc["P2_ZM87_V_D", "Sex"]
metadata_biomarkers.at['2_ZM_6', 'Sex'] = metadata_skin.loc["P2_ZM87_VI_D", "Sex"]
metadata_biomarkers.at['2_ZM_1', 'Age'] = metadata_skin.loc["P2_ZM87_I_D", "Age"]
metadata_biomarkers.at['2_ZM_2', 'Age'] = metadata_skin.loc["P2_ZM87_II_D", "Age"]
metadata_biomarkers.at['2_ZM_3', 'Age'] = metadata_skin.loc["P2_ZM87_III_D", "Age"]
metadata_biomarkers.at['2_ZM_4', 'Age'] = metadata_skin.loc["P2_ZM87_IV_D", "Age"]
metadata_biomarkers.at['2_ZM_5', 'Age'] = metadata_skin.loc["P2_ZM87_V_D", "Age"]
metadata_biomarkers.at['2_ZM_6', 'Age'] = metadata_skin.loc["P2_ZM87_VI_D", "Age"]
metadata_biomarkers.at['2_ZM_1', 'BMI'] = metadata_skin.loc["P2_ZM87_I_D", "BMI"]
metadata_biomarkers.at['2_ZM_2', 'BMI'] = metadata_skin.loc["P2_ZM87_II_D", "BMI"]
metadata_biomarkers.at['2_ZM_3', 'BMI'] = metadata_skin.loc["P2_ZM87_III_D", "BMI"]
metadata_biomarkers.at['2_ZM_4', 'BMI'] = metadata_skin.loc["P2_ZM87_IV_D", "BMI"]
metadata_biomarkers.at['2_ZM_5', 'BMI'] = metadata_skin.loc["P2_ZM87_V_D", "BMI"]
metadata_biomarkers.at['2_ZM_6', 'BMI'] = metadata_skin.loc["P2_ZM87_VI_D", "BMI"]
metadata_biomarkers.at['2_ZM_1', 'Disease_duration_years'] = metadata_skin.loc["P2_ZM87_I_D", "Disease_duration_years"]
metadata_biomarkers.at['2_ZM_2', 'Disease_duration_years'] = metadata_skin.loc["P2_ZM87_II_D", "Disease_duration_years"]
metadata_biomarkers.at['2_ZM_3', 'Disease_duration_years'] = metadata_skin.loc["P2_ZM87_III_D", "Disease_duration_years"]
metadata_biomarkers.at['2_ZM_4', 'Disease_duration_years'] = metadata_skin.loc["P2_ZM87_IV_D", "Disease_duration_years"]
metadata_biomarkers.at['2_ZM_5', 'Disease_duration_years'] = metadata_skin.loc["P2_ZM87_V_D", "Disease_duration_years"]
metadata_biomarkers.at['2_ZM_6', 'Disease_duration_years'] = metadata_skin.loc["P2_ZM87_VI_D", "Disease_duration_years"]
metadata_biomarkers.at['2_ZM_1', 'Age_at_diagnosis'] = metadata_skin.loc["P2_ZM87_I_D", "Age_at_diagnosis"]
metadata_biomarkers.at['2_ZM_2', 'Age_at_diagnosis'] = metadata_skin.loc["P2_ZM87_II_D", "Age_at_diagnosis"]
metadata_biomarkers.at['2_ZM_3', 'Age_at_diagnosis'] = metadata_skin.loc["P2_ZM87_III_D", "Age_at_diagnosis"]
metadata_biomarkers.at['2_ZM_4', 'Age_at_diagnosis'] = metadata_skin.loc["P2_ZM87_IV_D", "Age_at_diagnosis"]
metadata_biomarkers.at['2_ZM_5', 'Age_at_diagnosis'] = metadata_skin.loc["P2_ZM87_V_D", "Age_at_diagnosis"]
metadata_biomarkers.at['2_ZM_6', 'Age_at_diagnosis'] = metadata_skin.loc["P2_ZM87_VI_D", "Age_at_diagnosis"]
metadata_biomarkers.at['2_ZM_1', 'Disease_severity_scale4'] = metadata_skin.loc["P2_ZM87_I_D", "Disease_severity_scale4"]
metadata_biomarkers.at['2_ZM_6', 'Disease_severity_scale4'] = metadata_skin.loc["P2_ZM87_VI_D", "Disease_severity_scale4"]
metadata_biomarkers.at['2_ZM_1', 'HBI'] = metadata_skin.loc["P2_ZM87_I_D", "HBI"]
metadata_biomarkers.at['2_ZM_6', 'HBI'] = metadata_skin.loc["P2_ZM87_VI_D", "HBI"]
metadata_biomarkers.at['2_ZM_1', 'CRP'] = 13.5
metadata_biomarkers.at['2_ZM_1', 'WBC'] = 8.5
metadata_biomarkers.at['2_ZM_1', 'PLT'] = 426
metadata_biomarkers.at['2_ZM_1', 'Ferritin'] = 57.95
metadata_biomarkers.at['2_ZM_1', 'Hb'] = 153
metadata_biomarkers.at['2_ZM_1', 'FC'] = 1278
metadata_biomarkers.at['2_ZM_6', 'CRP'] = 2.3
metadata_biomarkers.at['2_ZM_6', 'WBC'] = 6.5
metadata_biomarkers.at['2_ZM_6', 'PLT'] = 350
metadata_biomarkers.at['2_ZM_6', 'Ferritin'] = 41.45
metadata_biomarkers.at['2_ZM_6', 'Hb'] = 156
metadata_biomarkers.at['2_ZM_6', 'FC'] = 469
metadata_biomarkers.at['18_SJ_6', 'Disease_status'] = metadata_biomarkers.loc["18_SJ_1", "Disease_status"]
metadata_biomarkers.at['18_SJ_6', 'Dg'] = metadata_biomarkers.loc["18_SJ_1", "Dg"]
metadata_biomarkers.at['18_SJ_6', 'Sex'] = metadata_biomarkers.loc["18_SJ_1", "Sex"]
metadata_biomarkers.at['18_SJ_6', 'Age'] = metadata_biomarkers.loc["18_SJ_1", "Age"]
metadata_biomarkers.at['18_SJ_6', 'BMI'] = 29.7
metadata_biomarkers.at['18_SJ_6', 'Disease_duration_years'] = metadata_biomarkers.loc["18_SJ_1", "Disease_duration_years"]
metadata_biomarkers.at['18_SJ_6', 'Age_at_diagnosis'] = metadata_biomarkers.loc["18_SJ_1", "Age_at_diagnosis"]
metadata_biomarkers.at['18_SJ_6', 'Disease_severity_scale4'] = "remission"
metadata_biomarkers.at['18_SJ_6', 'HBI'] = 3
metadata_biomarkers.at['18_SJ_6', 'CRP'] = 6.8
metadata_biomarkers.at['18_SJ_6', 'WBC'] = 7.7
metadata_biomarkers.at['18_SJ_6', 'PLT'] = 469
metadata_biomarkers.at['18_SJ_6', 'Ferritin'] = 211.11
metadata_biomarkers.at['18_SJ_6', 'Hb'] = 156
metadata_biomarkers.at['18_SJ_6', 'FC'] = 76
metadata_biomarkers.at['30_KM_1', 'Disease_status'] = metadata_biomarkers.loc["30_KM_2", "Disease_status"]
metadata_biomarkers.at['30_KM_1', 'Dg'] = metadata_biomarkers.loc["30_KM_2", "Dg"]
metadata_biomarkers.at['30_KM_1', 'Sex'] = metadata_biomarkers.loc["30_KM_2", "Sex"]
metadata_biomarkers.at['30_KM_1', 'Age'] = 46
metadata_biomarkers.at['30_KM_1', 'BMI'] = 27.6
metadata_biomarkers.at['30_KM_1', 'Disease_duration_years'] = metadata_biomarkers.loc["30_KM_2", "Disease_duration_years"]
metadata_biomarkers.at['30_KM_1', 'Age_at_diagnosis'] = metadata_biomarkers.loc["30_KM_2", "Age_at_diagnosis"]
metadata_biomarkers.at['30_KM_1', 'Disease_severity_scale4'] = "moderate"
metadata_biomarkers.at['30_KM_1', 'HBI'] = 8
metadata_biomarkers.at['30_KM_1', 'CRP'] = 3.7
metadata_biomarkers.at['30_KM_1', 'WBC'] = 6.9
metadata_biomarkers.at['30_KM_1', 'PLT'] = 422
metadata_biomarkers.at['30_KM_1', 'Ferritin'] = 4.16
metadata_biomarkers.at['30_KM_1', 'Hb'] = 113
metadata_biomarkers.at['30_KM_1', 'FC'] = 1678

# add values for HC26, HC28
metadata_biomarkers_hc.at['K26_1', 'Disease_status'] = "HC"
metadata_biomarkers_hc.at['K26_1', 'Dg'] = "HC"
metadata_biomarkers_hc.at['K28_1', 'Disease_status'] = "HC"
metadata_biomarkers_hc.at['K28_1', 'Dg'] = metadata_skin.loc["K28_KZ83_I_D", "Dg"]

# change data type
metadata_biomarkers['Age'] = metadata_biomarkers['Age'].astype(float)
metadata_biomarkers['BMI'] = metadata_biomarkers['BMI'].astype(float)
metadata_biomarkers['Disease_duration_years'] = metadata_biomarkers['Disease_duration_years'].astype(float)
metadata_biomarkers['Age_at_diagnosis'] = metadata_biomarkers['Age_at_diagnosis'].astype(float)
metadata_biomarkers['CRP'] = metadata_biomarkers['CRP'].astype(float)
metadata_biomarkers['WBC'] = metadata_biomarkers['WBC'].astype(float)
metadata_biomarkers['PLT'] = metadata_biomarkers['PLT'].astype(float)
metadata_biomarkers['Ferritin'] = metadata_biomarkers['Ferritin'].astype(float)
metadata_biomarkers['Hb'] = metadata_biomarkers['Hb'].astype(float)
metadata_biomarkers['FC'] = metadata_biomarkers['FC'].astype(float)
metadata_biomarkers_hc['Age'] = metadata_biomarkers_hc['Age'].astype(float)
metadata_biomarkers_hc['BMI'] = metadata_biomarkers_hc['BMI'].astype(float)


In [25]:
# baseline
ust_biomarkers_cd_0 = metadata_biomarkers[(metadata_biomarkers["Dg"]=="CD") & (metadata_biomarkers["Week"]=="week_0")]
ust_biomarkers_uc_0 = metadata_biomarkers[(metadata_biomarkers["Dg"]=="UC") & (metadata_biomarkers["Week"]=="week_0")]
ust_biomarkers_0 = metadata_biomarkers[metadata_biomarkers["Week"]=="week_0"]

# endpoint
ust_biomarkers_cd_40 = metadata_biomarkers[(metadata_biomarkers["Dg"]=="CD") & (metadata_biomarkers["Week"]=="week_40")]
ust_biomarkers_uc_40 = metadata_biomarkers[(metadata_biomarkers["Dg"]=="UC") & (metadata_biomarkers["Week"]=="week_40")]
ust_biomarkers_40 = metadata_biomarkers[metadata_biomarkers["Week"]=="week_40"]

# healthy controls
ust_biomarkers_hc_0 = metadata_biomarkers_hc[metadata_biomarkers_hc["Week"]=="week_0"]

In [26]:
# IBD and HC counts
print("Number of IBD patients at baseline (week 0) = ", metadata_biomarkers[(metadata_biomarkers['Disease_status']=="IBD") & (metadata_biomarkers['Week']=="week_0")]["Patient"].nunique())
print("Number of IBD patients at endpoint (week 40) = ", metadata_biomarkers[(metadata_biomarkers['Disease_status']=="IBD") & (metadata_biomarkers['Week']=="week_40")]["Patient"].nunique())
print("Number of IBD females  at baseline (week 0) = ", metadata_biomarkers[(metadata_biomarkers['Sex']=="F") & (metadata_biomarkers['Week']=="week_0")]["Patient"].nunique())
print("Number of IBD males  at baseline (week 0) = ", metadata_biomarkers[(metadata_biomarkers['Sex']=="M") & (metadata_biomarkers['Week']=="week_0")]["Patient"].nunique())
print("Number of IBD females  at endpoint (week 40) = ", metadata_biomarkers[(metadata_biomarkers['Sex']=="F") & (metadata_biomarkers['Week']=="week_40")]["Patient"].nunique())
print("Number of IBD males  at endpoint (week 40) = ", metadata_biomarkers[(metadata_biomarkers['Sex']=="M") & (metadata_biomarkers['Week']=="week_40")]["Patient"].nunique())
print("Number of CD patients", metadata_biomarkers[(metadata_biomarkers['Dg']=="CD")]["Patient"].nunique())
print("Number of UC patients", metadata_biomarkers[(metadata_biomarkers['Dg']=="UC")]["Patient"].nunique())
print("Number of CD patients at baseline (week 0)", metadata_biomarkers[(metadata_biomarkers['Dg']=="CD") & (metadata_biomarkers['Week']=="week_0")]["Patient"].nunique())
print("Number of UC patients at baseline (week 0)", metadata_biomarkers[(metadata_biomarkers['Dg']=="UC") & (metadata_biomarkers['Week']=="week_0")]["Patient"].nunique())
print("Number of CD patients at endpoint (week 40)", metadata_biomarkers[(metadata_biomarkers['Dg']=="CD") & (metadata_biomarkers['Week']=="week_40")]["Patient"].nunique())
print("Number of UC patients at endpoint (week 40)", metadata_biomarkers[(metadata_biomarkers['Dg']=="UC") & (metadata_biomarkers['Week']=="week_40")]["Patient"].nunique())
print("Number of remitters at baseline (week 0) = ", metadata_biomarkers[(metadata_biomarkers['Disease_severity_scale4']=="remission") & (metadata_biomarkers['Week']=="week_0")]["Patient"].nunique())
print("Number of remitters at endpoint (week 40) = ", metadata_biomarkers[(metadata_biomarkers['Disease_severity_scale4']=="remission") & (metadata_biomarkers['Week']=="week_40")]["Patient"].nunique())
print("Number of HC = ", metadata_biomarkers_hc[metadata_biomarkers_hc['Disease_status']=="HC"]["Patient"].nunique())
print("Number of HC females entering the study = ", metadata_biomarkers_hc[(metadata_biomarkers_hc['Sex']=="F")]["Patient"].nunique())
print("Number of HC males entering the study = ", metadata_biomarkers_hc[(metadata_biomarkers_hc['Sex']=="M")]["Patient"].nunique())

Number of IBD patients at baseline (week 0) =  9
Number of IBD patients at endpoint (week 40) =  8
Number of IBD females  at baseline (week 0) =  4
Number of IBD males  at baseline (week 0) =  5
Number of IBD females  at endpoint (week 40) =  4
Number of IBD males  at endpoint (week 40) =  4
Number of CD patients 7
Number of UC patients 2
Number of CD patients at baseline (week 0) 7
Number of UC patients at baseline (week 0) 2
Number of CD patients at endpoint (week 40) 6
Number of UC patients at endpoint (week 40) 2
Number of remitters at baseline (week 0) =  3
Number of remitters at endpoint (week 40) =  5
Number of HC =  2
Number of HC females entering the study =  0
Number of HC males entering the study =  0


In [27]:
## Anthropometric data HC
print(f'HC Age\n{ust_biomarkers_hc_0.Age.quantile([0.25,0.5,0.75])}')
print(f'HC BMI\n{ust_biomarkers_hc_0.BMI.quantile([0.25,0.5,0.75])}')

## Anthropometric data IBD
# IBD baseline
print(f'IBD Age\n{ust_biomarkers_0.Age.quantile([0.25,0.5,0.75])}')
print(f'IBD BMI\n{ust_biomarkers_0.BMI.quantile([0.25,0.5,0.75])}')
print(f'IBD Disease duration years\n{ust_biomarkers_0.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'IBD Age at diagnosis\n{ust_biomarkers_0.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')
# IBD endpoint
print(f'IBD Age\n{ust_biomarkers_40.Age.quantile([0.25,0.5,0.75])}')
print(f'IBD BMI\n{ust_biomarkers_40.BMI.quantile([0.25,0.5,0.75])}')
print(f'IBD Disease duration years\n{ust_biomarkers_40.Disease_duration_years.quantile([0.25,0.5,0.75])}')
print(f'IBD Age at diagnosis\n{ust_biomarkers_40.Age_at_diagnosis.quantile([0.25,0.5,0.75])}')

HC Age
0.25   NaN
0.50   NaN
0.75   NaN
Name: Age, dtype: float64
HC BMI
0.25   NaN
0.50   NaN
0.75   NaN
Name: BMI, dtype: float64
IBD Age
0.25    33.0
0.50    40.0
0.75    46.0
Name: Age, dtype: float64
IBD BMI
0.25    19.57
0.50    25.40
0.75    27.40
Name: BMI, dtype: float64
IBD Disease duration years
0.25    10.0
0.50    11.0
0.75    14.0
Name: Disease_duration_years, dtype: float64
IBD Age at diagnosis
0.25    25.0
0.50    32.0
0.75    39.0
Name: Age_at_diagnosis, dtype: float64
IBD Age
0.25    39.0
0.50    41.5
0.75    49.5
Name: Age, dtype: float64
IBD BMI
0.25    24.2625
0.50    25.8000
0.75    27.7000
Name: BMI, dtype: float64
IBD Disease duration years
0.25     9.50
0.50    10.50
0.75    14.25
Name: Disease_duration_years, dtype: float64
IBD Age at diagnosis
0.25    26.5
0.50    34.5
0.75    40.0
Name: Age_at_diagnosis, dtype: float64


In [28]:
# Clinical data baseline
# CD baseline
ust_biomarkers_cd_0['HBI'] = ust_biomarkers_cd_0['HBI'].astype(float)
print(f'CD HBI\n{ust_biomarkers_cd_0.HBI.quantile([0.25,0.5,0.75])}')

# UC baseline
ust_biomarkers_uc_0['pMayo'] = ust_biomarkers_uc_0['pMayo'].astype(float)
print(f'UC pMayo\n{ust_biomarkers_uc_0.pMayo.quantile([0.25,0.5,0.75])}')

# IBD baseline
print(f'IBD CRP\n{ust_biomarkers_0.CRP.quantile([0.25,0.5,0.75])}')
print(f'IBD WBC\n{ust_biomarkers_0.WBC.quantile([0.25,0.5,0.75])}')
print(f'IBD PLT\n{ust_biomarkers_0.PLT.quantile([0.25,0.5,0.75])}')
print(f'IBD Ferritin\n{ust_biomarkers_0.Ferritin.quantile([0.25,0.5,0.75])}')
print(f'IBD Hb\n{ust_biomarkers_0.Hb.quantile([0.25,0.5,0.75])}')
print(f'IBD FC\n{ust_biomarkers_0.FC.quantile([0.25,0.5,0.75])}')

# HC baseline
print(f'HC CRP\n{ust_biomarkers_hc_0.CRP.quantile([0.25,0.5,0.75])}')
print(f'HC WBC\n{ust_biomarkers_hc_0.WBC.quantile([0.25,0.5,0.75])}')
print(f'HC PLT\n{ust_biomarkers_hc_0.PLT.quantile([0.25,0.5,0.75])}')
print(f'HC Hb\n{ust_biomarkers_hc_0.Hb.quantile([0.25,0.5,0.75])}')

CD HBI
0.25    3.5
0.50    5.0
0.75    7.5
Name: HBI, dtype: float64
UC pMayo
0.25    5.25
0.50    6.50
0.75    7.75
Name: pMayo, dtype: float64
IBD CRP
0.25     3.8
0.50     6.4
0.75    10.3
Name: CRP, dtype: float64
IBD WBC
0.25    6.9
0.50    7.8
0.75    9.1
Name: WBC, dtype: float64
IBD PLT
0.25    374.0
0.50    402.0
0.75    426.0
Name: PLT, dtype: float64
IBD Ferritin
0.25    15.04
0.50    20.47
0.75    33.16
Name: Ferritin, dtype: float64
IBD Hb
0.25    115.0
0.50    126.0
0.75    134.0
Name: Hb, dtype: float64
IBD FC
0.25     380.0
0.50    1278.0
0.75    1678.0
Name: FC, dtype: float64
HC CRP
0.25   NaN
0.50   NaN
0.75   NaN
Name: CRP, dtype: float64
HC WBC
0.25   NaN
0.50   NaN
0.75   NaN
Name: WBC, dtype: float64
HC PLT
0.25   NaN
0.50   NaN
0.75   NaN
Name: PLT, dtype: float64
HC Hb
0.25   NaN
0.50   NaN
0.75   NaN
Name: Hb, dtype: float64


<ipython-input-28-7a10f8eccaf6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_biomarkers_cd_0['HBI'] = ust_biomarkers_cd_0['HBI'].astype(float)
<ipython-input-28-7a10f8eccaf6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_biomarkers_uc_0['pMayo'] = ust_biomarkers_uc_0['pMayo'].astype(float)


In [29]:
# Clinical data endpoint
# CD endpoint
ust_biomarkers_cd_40['HBI'] = ust_biomarkers_cd_40['HBI'].astype(float)
print(f'CD HBI\n{ust_biomarkers_cd_40.HBI.quantile([0.25,0.5,0.75])}')

# UC endpoint
ust_biomarkers_uc_40['pMayo'] = ust_biomarkers_uc_40['pMayo'].astype(float)
print(f'UC pMayo\n{ust_biomarkers_uc_40.pMayo.quantile([0.25,0.5,0.75])}')

# IBD endpoint
print(f'IBD CRP\n{ust_biomarkers_40.CRP.quantile([0.25,0.5,0.75])}')
print(f'IBD WBC\n{ust_biomarkers_40.WBC.quantile([0.25,0.5,0.75])}')
print(f'IBD PLT\n{ust_biomarkers_40.PLT.quantile([0.25,0.5,0.75])}')
print(f'IBD Ferritin\n{ust_biomarkers_40.Ferritin.quantile([0.25,0.5,0.75])}')
print(f'IBD Hb\n{ust_biomarkers_40.Hb.quantile([0.25,0.5,0.75])}')
print(f'IBD FC\n{ust_biomarkers_40.FC.quantile([0.25,0.5,0.75])}')

CD HBI
0.25    3.0
0.50    3.0
0.75    4.5
Name: HBI, dtype: float64
UC pMayo
0.25    1.25
0.50    1.50
0.75    1.75
Name: pMayo, dtype: float64
IBD CRP
0.25    2.225
0.50    2.650
0.75    4.250
Name: CRP, dtype: float64
IBD WBC
0.25    6.875
0.50    7.250
0.75    8.250
Name: WBC, dtype: float64
IBD PLT
0.25    260.00
0.50    320.50
0.75    379.75
Name: PLT, dtype: float64
IBD Ferritin
0.25     7.085
0.50    14.375
0.75    37.655
Name: Ferritin, dtype: float64
IBD Hb
0.25    121.25
0.50    128.50
0.75    156.00
Name: Hb, dtype: float64
IBD FC
0.25     61.00
0.50    182.50
0.75    483.75
Name: FC, dtype: float64


<ipython-input-29-d0532c75efb6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_biomarkers_cd_40['HBI'] = ust_biomarkers_cd_40['HBI'].astype(float)
<ipython-input-29-d0532c75efb6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ust_biomarkers_uc_40['pMayo'] = ust_biomarkers_uc_40['pMayo'].astype(float)


In [30]:
# modify for pairwise tests
ust_biomarkers_0w = ust_biomarkers_0.drop("35_AJ_1")
ust_biomarkers_cd_0w = ust_biomarkers_cd_0.drop("35_AJ_1")

# calculate pairwise comparison between week 0 and week 40 in IBD patients
print(f'HBI {stats.wilcoxon(ust_biomarkers_cd_0w["HBI"], ust_biomarkers_cd_40["HBI"])}')
print(f'pMayo {stats.wilcoxon(ust_biomarkers_uc_0["pMayo"], ust_biomarkers_uc_40["pMayo"])}')
print(f'CRP {stats.wilcoxon(ust_biomarkers_0w["CRP"], ust_biomarkers_40["CRP"])}')
print(f'WBC {stats.wilcoxon(ust_biomarkers_0w["WBC"], ust_biomarkers_40["WBC"])}')
print(f'PLT {stats.wilcoxon(ust_biomarkers_0w["PLT"], ust_biomarkers_40["PLT"])}')
print(f'Ferritin {stats.wilcoxon(ust_biomarkers_0w["Ferritin"], ust_biomarkers_40["Ferritin"])}')
print(f'Hb {stats.wilcoxon(ust_biomarkers_0w["Hb"], ust_biomarkers_40["Hb"])}')
print(f'FC {stats.wilcoxon(ust_biomarkers_0w["FC"], ust_biomarkers_40["FC"])}')

# correct for multiple hypothesis testing
wilcox_pval = []
wilcox_pval.append(stats.wilcoxon(ust_biomarkers_cd_0w["HBI"], ust_biomarkers_cd_40["HBI"])[1])
wilcox_pval.append(stats.wilcoxon(ust_biomarkers_uc_0["pMayo"], ust_biomarkers_uc_40["pMayo"])[1])
wilcox_pval.append(stats.wilcoxon(ust_biomarkers_0w["CRP"], ust_biomarkers_40["CRP"])[1])
wilcox_pval.append(stats.wilcoxon(ust_biomarkers_0w["WBC"], ust_biomarkers_40["WBC"])[1])
wilcox_pval.append(stats.wilcoxon(ust_biomarkers_0w["PLT"], ust_biomarkers_40["PLT"])[1])
wilcox_pval.append(stats.wilcoxon(ust_biomarkers_0w["Ferritin"], ust_biomarkers_40["Ferritin"])[1])
wilcox_pval.append(stats.wilcoxon(ust_biomarkers_0w["Hb"], ust_biomarkers_40["Hb"])[1])
wilcox_pval.append(stats.wilcoxon(ust_biomarkers_0w["FC"], ust_biomarkers_40["FC"])[1])
print(f'Bonferroni corrected pvals CRP, WBC, PLT, Ferritin, Hb, FC = {sm.multitest.multipletests(wilcox_pval, alpha=0.05, method="bonferroni")[1]}')

HBI WilcoxonResult(statistic=3.0, pvalue=0.4614509878333607)
pMayo WilcoxonResult(statistic=0.0, pvalue=0.5)
CRP WilcoxonResult(statistic=4.0, pvalue=0.0546875)
WBC WilcoxonResult(statistic=6.5, pvalue=0.1484375)
PLT WilcoxonResult(statistic=6.0, pvalue=0.17629637444051116)
Ferritin WilcoxonResult(statistic=17.0, pvalue=0.9453125)
Hb WilcoxonResult(statistic=11.0, pvalue=0.3828125)
FC WilcoxonResult(statistic=5.0, pvalue=0.078125)
Bonferroni corrected pvals CRP, WBC, PLT, Ferritin, Hb, FC = [1.     1.     0.4375 1.     1.     1.     1.     0.625 ]


In [31]:
# calculate comparison between HC and IBD
print(f'CRP {stats.mannwhitneyu(ust_biomarkers_0["CRP"], ust_biomarkers_hc_0["CRP"])}')
print(f'WBC {stats.mannwhitneyu(ust_biomarkers_0["WBC"], ust_biomarkers_hc_0["WBC"])}')
print(f'PLT {stats.mannwhitneyu(ust_biomarkers_0["PLT"], ust_biomarkers_hc_0["PLT"])}')
print(f'Hb {stats.mannwhitneyu(ust_biomarkers_0["Hb"], ust_biomarkers_hc_0["Hb"])}')

# correct for multiple hypothesis testing
mannwu_pval = []
mannwu_pval.append(stats.mannwhitneyu(ust_biomarkers_0["CRP"], ust_biomarkers_hc_0["CRP"])[1])
mannwu_pval.append(stats.mannwhitneyu(ust_biomarkers_0["WBC"], ust_biomarkers_hc_0["WBC"])[1])
mannwu_pval.append(stats.mannwhitneyu(ust_biomarkers_0["PLT"], ust_biomarkers_hc_0["PLT"])[1])
mannwu_pval.append(stats.mannwhitneyu(ust_biomarkers_0["Hb"], ust_biomarkers_hc_0["Hb"])[1])
print(f'Bonferroni corrected pvals CRP, WBC, PLT, Hb = {sm.multitest.multipletests(mannwu_pval, alpha=0.05, method="bonferroni")[1]}')
sm.multitest.multipletests

CRP MannwhitneyuResult(statistic=0.0, pvalue=2.0153596983127664e-06)
WBC MannwhitneyuResult(statistic=0.0, pvalue=2.0153596983127664e-06)
PLT MannwhitneyuResult(statistic=0.0, pvalue=2.0153596983127664e-06)
Hb MannwhitneyuResult(statistic=0.0, pvalue=2.0153596983127664e-06)
Bonferroni corrected pvals CRP, WBC, PLT, Hb = [8.06143879e-06 8.06143879e-06 8.06143879e-06 8.06143879e-06]


<function statsmodels.stats.multitest.multipletests(pvals, alpha=0.05, method='hs', is_sorted=False, returnsorted=False)>

## Antibody levels

In [32]:
# same as biomarkers